In [34]:
import pandas as pd
import numpy as np

from pmdarima import auto_arima
from statsmodels.tsa.statespace.sarimax import SARIMAX


In [39]:
# list for loop in test data 
test_list = ["test_50hz",
"test_amprion",
"test_tennet",
"test_transnet"]

In [ ]:
for dframe in test_list:
    df = pd.read_parquet(f"data/{dframe}.parquet")
    startt =time.time()
    # Initialize relevant lists
    results = []
    iteration = 1
    

    # Sample data series
    CI_series = df["target"]
    CI_series.index = CI_series.index * np.timedelta64(1, 'h') + pd.Timestamp('2015-01-05 00:00:00+0100', tz='Europe/Berlin')
    CI_series.index.freq = 'h'

    # Define windows
    train_window_size = 28 * 24
    forecast_horizon = 24

    # Rolling window approach
    for start in range(0, len(CI_series) - train_window_size - forecast_horizon + 1, forecast_horizon):

        # Define the train and test windows
        train_window = CI_series[start:start + train_window_size]
        test_window = CI_series[start + train_window_size:start + train_window_size + forecast_horizon]

        # Define parameter and seasonal order (Lowry 2018)
        model_order = (4,0,0)
        seasonal_order = (4,0,0,24)

        model = SARIMAX(train_window,
                        order=model_order,
                        seasonal_order=seasonal_order,
                        enforce_stationarity=False,
                        enforce_invertibility=False,
                        simple_differencing=True,
                        mle_regression=False
                        )
        print(f"iteration: {iteration}, from: {start}, to: {start + train_window_size + forecast_horizon}")
        start= time.time()
        print(f"start: {start}")
        model_fit = model.fit()
        start= time.time()
        print(f"end - start: {end - start}")
        # Forecast next 24 hours
        forecast = model_fit.forecast(steps=forecast_horizon,
                                      method='cg',
                                      optim_complex_step=False,
                                      disp=0)

        # Record results for each forecasted point
        for i, (fcst, actual) in enumerate(zip(forecast, test_window)):
            date = test_window.index[i]
            results.append({
                "date": date,
                "forecast": fcst,
                "actual": actual,
                "iteration": iteration,
                "start_train_window": train_window.index[0],
                "end_train_window": train_window.index[-1],
                "start_test_window": test_window.index[0],
                "end_test_window": test_window.index[-1]
            })

        # Increment iteration count
        #iteration += 1
        #if iteration % 20 == 0:
            #print(f"{dframe} - iteration:{iteration}")
    # Create a DataFrame from the collected results
    results_df = pd.DataFrame(results)

    # Set "date" column as the index
    results_df.set_index("date", inplace=True)

    # Save results

    results_df.to_parquet(f"data/sarima_results_{dframe}_400_40024.parquet")


iteration: 1, from: 0, to: 696
start: 1715407069.3903506
RUNNING THE L-BFGS-B CODE

           * * *

Machine precision = 2.220D-16
 N =            9     M =           10

At X0         0 variables are exactly at the bounds

At iterate    0    f=  3.70315D+00    |proj g|=  9.05474D-01


 This problem is unconstrained.



At iterate    5    f=  3.56291D+00    |proj g|=  1.19456D-01

At iterate   10    f=  3.54964D+00    |proj g|=  2.62453D-02

At iterate   15    f=  3.54894D+00    |proj g|=  4.74290D-02

At iterate   20    f=  3.54881D+00    |proj g|=  1.46622D-02

At iterate   25    f=  3.54752D+00    |proj g|=  2.42212D-02

At iterate   30    f=  3.54742D+00    |proj g|=  6.67874D-02

At iterate   35    f=  3.54538D+00    |proj g|=  1.41824D-01

At iterate   40    f=  3.54395D+00    |proj g|=  8.62911D-02

At iterate   45    f=  3.54264D+00    |proj g|=  1.42407D-01

At iterate   50    f=  3.54239D+00    |proj g|=  1.22298D-02

           * * *

Tit   = total number of iterations
Tnf   = total number of function evaluations
Tnint = total number of segments explored during Cauchy searches
Skip  = number of BFGS updates skipped
Nact  = number of active bounds at final generalized Cauchy point
Projg = norm of the final projected gradient
F     = final function value

           * * *

   N    Tit     Tn

/home/RDC/tramburc/.conda/envs/stock_tft/lib/python3.10/site-packages/statsmodels/base/model.py:607: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  warnings.warn("Maximum Likelihood optimization failed to "


end - start: -122177.24576425552
iteration: 2, from: 24, to: 720
start: 1715407281.8402836


...

/home/RDC/tramburc/.conda/envs/stock_tft/lib/python3.10/site-packages/statsmodels/tsa/statespace/representation.py:374: FutureWarning: Unknown keyword arguments: dict_keys(['method', 'optim_complex_step', 'disp']).Passing unknown keyword arguments will raise a TypeError beginning in version 0.15.
  warnings.warn(msg, FutureWarning)


RUNNING THE L-BFGS-B CODE

           * * *

Machine precision = 2.220D-16
 N =            9     M =           10

At X0         0 variables are exactly at the bounds

At iterate    0    f=  3.48821D+00    |proj g|=  1.30696D+00


 This problem is unconstrained.



At iterate    5    f=  3.34241D+00    |proj g|=  4.43143D-01

At iterate   10    f=  3.33926D+00    |proj g|=  2.20255D-01

At iterate   15    f=  3.33568D+00    |proj g|=  2.44215D-02

At iterate   20    f=  3.33547D+00    |proj g|=  1.57777D-01

At iterate   25    f=  3.33517D+00    |proj g|=  1.34909D-02

At iterate   30    f=  3.33510D+00    |proj g|=  7.16335D-03

At iterate   35    f=  3.33498D+00    |proj g|=  1.34161D-02

At iterate   40    f=  3.33449D+00    |proj g|=  6.23028D-02

At iterate   45    f=  3.33386D+00    |proj g|=  1.19291D-02

At iterate   50    f=  3.33383D+00    |proj g|=  3.72611D-02

           * * *

Tit   = total number of iterations
Tnf   = total number of function evaluations
Tnint = total number of segments explored during Cauchy searches
Skip  = number of BFGS updates skipped
Nact  = number of active bounds at final generalized Cauchy point
Projg = norm of the final projected gradient
F     = final function value

           * * *

   N    Tit     Tn

/home/RDC/tramburc/.conda/envs/stock_tft/lib/python3.10/site-packages/statsmodels/base/model.py:607: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  warnings.warn("Maximum Likelihood optimization failed to "
